# Autonomous Agent To: 
## Fetch, Analyze and Backtest Trading Stratergy on Forex pair

## Install the requiremed dependencies

In [2]:
!pip install -U --quiet langgraph langchain_openai tradermade

# 0. Setup 

In [5]:
import getpass
import os

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

_set_if_undefined("OPENAI_API_KEY")          # For calling LLM
_set_if_undefined("TRADERMADE_API_KEY")       # For forex data

import json
import pandas as pd
import tradermade as tm
from datetime import timedelta
import matplotlib.pyplot as plt

# Import agent dependencies
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, MessagesState, StateGraph, START
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import Literal

## Initialize LLM and set the TraderMade API key

In [11]:
llm = ChatOpenAI(model="gpt-4o")
tm.set_rest_api_key(os.environ["TRADERMADE_API_KEY"])

# Agent Functions
#### Each agent communicates using a chain prompt that pipes its output to the language model. 
#### Their outputs are then passed along in the workflow.

#### 1. Market Data Agent: Gather Forex Market Data and Compute Technical Signals

In [14]:
def market_data_agent(state: MessagesState):
    """Gathers and processes forex market data"""
    messages = state["messages"]
    params = messages[-1].additional_kwargs

    # Get historical forex data
    historical_data = get_forex_data(
        params["currency_pair"],
        params["start_date"],
        params["end_date"]
    )

    # Calculate trading signals
    signals = calculate_forex_signals(historical_data)

    # Create market data message
    message = HumanMessage(
        content=f"""
        EUR/USD Trading Signals:
        Current Price: {signals['current_price']:.5f}
        SMA 5: {signals['sma_5_curr']:.5f}
        SMA 5 Previous: {signals['sma_5_prev']:.5f}
        SMA 20: {signals['sma_20_curr']:.5f}
        SMA 20 Previous: {signals['sma_20_prev']:.5f}
        RSI: {signals['rsi']:.2f}
        """,
        name="market_data_agent",
    )

    return {"messages": messages + [message]}

#### 2. Quant Agent: Perform Forex Analysis Based on Technical Signals

In [18]:
def quant_agent(state: MessagesState):
    """Analyzes forex technical indicators"""
    last_message = state["messages"][-1]

    summary_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a forex trading expert analyzing EUR/USD.
                Analyze the technical signals and provide:
                - signal: bullish | bearish | neutral
                - confidence: <float 0-1>
                Consider forex market dynamics and 24-hour trading."""
            ),
            MessagesPlaceholder(variable_name="messages"),
            (
                "human",
                f"""Forex Analysis Data: {last_message.content}
                Provide only signal and confidence."""
            ),
        ]
    )

    chain = summary_prompt | llm
    result = chain.invoke(state).content
    return {"messages": state["messages"] + [
        HumanMessage(content=f"Forex Analysis: {result}", name="quant_agent")
    ]}

#### 3. Risk Management Agent: Evaluate Position Sizing and Risks

In [21]:
def risk_management_agent(state: MessagesState):
    """Forex-specific risk management"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]

    risk_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Forex Risk Manager: Evaluate position sizing considering
                - Max position: 1-5% of capital
                - Leverage (max 30:1)
                - Currency volatility
                Output: max_position_size, risk_score(1-10)"""
            ),
            MessagesPlaceholder(variable_name="messages"),
            (
                "human",
                f"""Analysis: {last_message.content}
                Portfolio: {portfolio['cash']:.2f} USD | Position: {portfolio['position']:.2f} EUR
                Provide max position size (EUR) and risk score."""
            ),
        ]
    )

    chain = risk_prompt | llm
    result = chain.invoke(state).content
    return {"messages": state["messages"] + [
        HumanMessage(content=f"Risk Assessment: {result}", name="risk_management")
    ]}

#### 4. Portfolio Management Agent: Decide Trade Actions and Lot Sizing

In [24]:
def portfolio_management_agent(state: MessagesState):
    """Executes forex trades with proper lot sizing"""
    portfolio = state["messages"][0].additional_kwargs["portfolio"]
    last_message = state["messages"][-1]

    portfolio_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Forex Portfolio Manager:
                - Standard lot = 100,000 EUR
                - Mini lot = 10,000 EUR
                - Micro lot = 1,000 EUR
                - Action: buy | sell | hold
                - Quantity: in EUR units
                Maintain 2% risk per trade"""
            ),
            MessagesPlaceholder(variable_name="messages"),
            (
                "human",
                f"""Risk Data: {last_message.content}
                Portfolio: {portfolio['cash']:.2f} USD | Position: {portfolio['position']:.2f} EUR
                Output action and quantity."""
            ),
        ]
    )

    chain = portfolio_prompt | llm
    result = chain.invoke(state).content
    return {"messages": [HumanMessage(content=result, name="portfolio_management")]}

#### 5. Agent Workflow Setup
*The workflow connects the above agents in sequence*

In [27]:
workflow = StateGraph(MessagesState)
workflow.add_node("market_data_agent", market_data_agent)
workflow.add_node("quant_agent", quant_agent)
workflow.add_node("risk_management_agent", risk_management_agent)
workflow.add_node("portfolio_management_agent", portfolio_management_agent)

workflow.add_edge(START, "market_data_agent")
workflow.add_edge("market_data_agent", "quant_agent")
workflow.add_edge("quant_agent", "risk_management_agent")
workflow.add_edge("risk_management_agent", "portfolio_management_agent")
workflow.add_edge("portfolio_management_agent", END)

app = workflow.compile()

#### 6. Forex Data and Signal Calculation Functions
*These functions fetch and process historical data via TraderMade’s timeseries endpoint*

In [30]:
def calculate_forex_signals(data: pd.DataFrame) -> dict:
    """Calculate forex technical indicators"""
    data['sma5'] = data['close'].rolling(5).mean()
    data['sma20'] = data['close'].rolling(20).mean()

    # RSI calculation
    delta = data['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))

    return {
        'current_price': data['close'].iloc[-1],
        'sma_5_curr': data['sma5'].iloc[-1],
        'sma_5_prev': data['sma5'].iloc[-2],
        'sma_20_curr': data['sma20'].iloc[-1],
        'sma_20_prev': data['sma20'].iloc[-2],
        'rsi': data['rsi'].iloc[-1]
    }

def get_forex_data(currency_pair: str, start: str, end: str) -> pd.DataFrame:
    """Fetch forex data from TraderMade"""
    try:
        data = tm.timeseries(
            currency=currency_pair,
            start=start,
            end=end,
            interval="daily",
            fields=["open", "high", "low", "close"]
        )

        if not data.get('quotes'):
            raise ValueError("No forex data returned")

        df = pd.DataFrame(data['quotes'])
        df['date'] = pd.to_datetime(df['date'])
        return df.set_index('date').sort_index()

    except Exception as e:
        raise Exception(f"TraderMade API Error: {str(e)}")

#### 7. Forex Backtester Class
*The backtester runs the agent workflow across a date range, parses its trade action output, executes trades, and records portfolio performance*

In [30]:
# 7. Final Corrected Forex Backtester
class ForexBacktester:
    def __init__(self, agent, currency_pair, start_date, end_date, initial_capital):
        self.agent = agent
        self.currency_pair = currency_pair
        self.start_date = start_date
        self.end_date = end_date
        self.initial_capital = initial_capital
        self.portfolio = {"cash": initial_capital, "position": 0.0}
        self.portfolio_values = []
        self.last_valid_price = 1.0  # Initialize with reasonable default

    def parse_action(self, output):
        try:
            # Robust parsing with error handling
            import re
            action = "hold"
            quantity = 0.0

            # Case-insensitive search for action and quantity
            action_match = re.search(r'action["\']?\s*:\s*["\']?(\w+)', output, re.IGNORECASE)
            quantity_match = re.search(r'quantity["\']?\s*:\s*([\d.]+)', output, re.IGNORECASE)

            if action_match:
                action = action_match.group(1).lower()
            if quantity_match:
                quantity = float(quantity_match.group(1))

            return action, max(0.0, quantity)  # Ensure non-negative quantity
        except:
            return "hold", 0.0

    def execute_trade(self, action, quantity, current_price):
        """Safe trade execution with validation"""
        if current_price <= 0:
            return 0.0

        quantity = round(quantity, 2)  # Prevent fractional units

        if action == "buy":
            max_affordable = self.portfolio["cash"] / current_price
            quantity = min(quantity, max_affordable)
            if quantity > 0:
                self.portfolio["position"] += quantity
                self.portfolio["cash"] -= quantity * current_price
                return quantity
        elif action == "sell":
            quantity = min(quantity, self.portfolio["position"])
            if quantity > 0:
                self.portfolio["position"] -= quantity
                self.portfolio["cash"] += quantity * current_price
                return quantity
        return 0.0

    def run_backtest(self):
        import pandas as pd
        from datetime import timedelta
        from langchain_core.messages import HumanMessage  # Ensure import

        dates = pd.date_range(self.start_date, self.end_date, freq="B")

        print("\nStarting Forex Backtest...")
        print(f"{'Date':<12} {'Action':<6} {'Quantity':>10} {'Price':>10} {'Cash':>12} {'Position':>12} {'Value':>12}")
        print("-" * 78)

        for current_date in dates:
            current_date_str = current_date.strftime("%Y-%m-%d")
            lookback_start = (current_date - timedelta(days=30)).strftime("%Y-%m-%d")

            # Initialize variables for this iteration
            action = "hold"
            executed_qty = 0.0
            current_price = self.last_valid_price  # Use last known price as fallback
            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            try:
                # Get agent decision
                agent_output = self.agent({
                    "messages": [HumanMessage(
                        content="Make a trading decision",
                        additional_kwargs={
                            "currency_pair": self.currency_pair,
                            "start_date": lookback_start,
                            "end_date": current_date_str,
                            "portfolio": self.portfolio.copy()
                        }
                    )]
                })
                final_output = agent_output["messages"][-1].content

                # Get market data with validation
                df = get_forex_data(self.currency_pair, lookback_start, current_date_str)
                if len(df) < 1:
                    raise ValueError("No data returned for date range")

                current_price = df.iloc[-1]['close']
                self.last_valid_price = current_price  # Update cache

                # Parse and execute trade
                action, quantity = self.parse_action(final_output)
                executed_qty = self.execute_trade(action, quantity, current_price)

            except Exception as e:
                print(f"Error processing {current_date_str}: {str(e)}")
                # Use cached price for valuation on error
                current_price = self.last_valid_price

            # Calculate total value using current/last price
            total_value = self.portfolio["cash"] + self.portfolio["position"] * current_price

            # Record portfolio state
            self.portfolio_values.append({
                "Date": current_date.date(),
                "Portfolio Value": total_value
            })

            # Print daily status with error recovery
            print(f"{current_date_str} "
                  f"{action:<6} {executed_qty:>10.2f} {current_price:>10.5f} "
                  f"{self.portfolio['cash']:>12.2f} {self.portfolio['position']:>12.2f} {total_value:>12.2f}")

    def analyze_performance(self):
        import pandas as pd
        import matplotlib.pyplot as plt

        if not self.portfolio_values:
            print("No backtest results to analyze")
            return None

        # Create DataFrame with validation
        performance_df = pd.DataFrame(self.portfolio_values)
        performance_df['Date'] = pd.to_datetime(performance_df['Date'])
        performance_df.set_index('Date', inplace=True)

        # Handle missing dates and forward fill
        all_dates = pd.date_range(start=self.start_date, end=self.end_date, freq='B')
        performance_df = performance_df.reindex(all_dates, method='ffill')

        # Calculate metrics
        initial = self.initial_capital
        final = performance_df['Portfolio Value'].iloc[-1]
        total_return = (final - initial) / initial

        print(f"\nBacktest Results ({self.currency_pair})")
        print(f"Initial Capital: ${initial:,.2f}")
        print(f"Final Value: ${final:,.2f}")
        print(f"Total Return: {total_return*100:.2f}%")

        # Plot performance
        performance_df["Portfolio Value"].plot(title="Forex Portfolio Performance", figsize=(12,6))
        plt.ylabel("Value (USD)")
        plt.show()

        return performance_df

# 8. Run Forex Backtest

In [ ]:
# 8. Agent Runner Function with Error Handling
def run_forex_agent(input_state):
    try:
        return app.invoke(input_state)
    except Exception as e:
        print(f"Agent Error: {str(e)}")
        return {"messages": [HumanMessage(content="hold|0", name="error_recovery")]}

# Run the backtest with final corrections
forex_backtester = ForexBacktester(
    agent=run_forex_agent,
    currency_pair="EURUSD",
    start_date="2025-01-01",
    end_date="2025-01-31",
    initial_capital=100000
)

forex_backtester.run_backtest()
forex_backtester.analyze_performance()


Starting Forex Backtest...
Date         Action   Quantity      Price         Cash     Position        Value
------------------------------------------------------------------------------
Agent Error: TraderMade API Error: No forex data returned
Error processing 2025-01-01: TraderMade API Error: No forex data returned
2025-01-01 hold         0.00    1.00000    100000.00         0.00    100000.00
Agent Error: TraderMade API Error: No forex data returned
Error processing 2025-01-02: TraderMade API Error: No forex data returned
2025-01-02 hold         0.00    1.00000    100000.00         0.00    100000.00
Agent Error: TraderMade API Error: HTTPSConnectionPool(host='marketdata.tradermade.com', port=443): Max retries exceeded with url: /api/v1/timeseries?currency=EURUSD&api_key=CEetCjNbCpjv3KFLo1iu&start_date=2024-12-04&interval=daily&end_date=2025-01-03&period=15&format=split (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1738caf60>: Failed to resolve 'mark

##### Debugging API key validity

In [32]:
# Check API key validity
import tradermade as tm
tm.set_rest_api_key(os.environ["TRADERMADE_API_KEY"])
print(tm.live(currency='EURUSD'))  # Should return current price

  instrument           timestamp     bid      mid      ask
0     EURUSD 2025-02-04 07:59:29  1.0311  1.03117  1.03123
